In [2]:
# In a Jupyter Notebook cell
%load_ext autoreload
%autoreload 2

from mirage.index import WordCountingChunkingAlgorithm, FolderRawStorage, SQLiteChunkStorage, RAMChunkStorage
from mirage.embedders import HuggingFaceEmbedder, TextNormalizer
import os
import pickle

In [4]:
import logging
logging.basicConfig(level=logging.ERROR)


documents = FolderRawStorage('data')
chunks = RAMChunkStorage()
algorithm = WordCountingChunkingAlgorithm(documents, chunks, words_amount=100)
algorithm.execute(visualize=True)



13

In [5]:
index = chunks.get_indexes()[0]
chunks[index]

'Приказ Росавиации от 28.12.2022 N 970-П\n"Об утверждении Норм летной годности\nгражданских легких самолетов НЛГ 23"\nДокумент предоставлен КонсультантПлюс\nwww.consultant.ru\nДата сохранения: 09.01.2024\nПриказ Росавиации от 28.12.2022 N 970-П\nДокумент предоставленКонсультантПлюс\n"Об утверждении Норм летной годности гражданских\nДата сохранения: 09.01.2024\nлегких самолетов НЛГ 23"\nМИНИСТЕРСТВО ТРАНСПОРТА РОССИЙСКОЙ ФЕДЕРАЦИИ\nФЕДЕРАЛЬНОЕ АГЕНТСТВО ВОЗДУШНОГО ТРАНСПОРТА\nПРИКАЗ\nот 28 декабря 2022 г. N 970-П\nОБ УТВЕРЖДЕНИИ НОРМ\nЛЕТНОЙ ГОДНОСТИ ГРАЖДАНСКИХ ЛЕГКИХ САМОЛЕТОВ НЛГ 23\nВ соответствии с пунктом 2 статьи 35 Воздушного кодекса Российской Федерации и подпунктами\n5.14.4 и 9.9 Положения о Федеральном агентстве воздушного транспорта, утвержденного постановлением\nПравительства Российской Федерации от 30 июля 2004 г. N 396, приказываю:\n1. Утвердить прилагаемыеНормы летной годности гражданских легких самолетов НЛГ 23.\n2. Настоящий приказ вступает в силу'

In [6]:
from mirage.embedders.BowEmbedder import BowEmbedder

emb = HuggingFaceEmbedder(normalizer = True)
emb.fit(chunks)

In [7]:
emb.get_dimensionality()


512

In [8]:
from mirage.index.vector_index.ram_vector_index import L2RAMVectorIndex

vector_index = L2RAMVectorIndex(dimensionality = emb.get_dimensionality())
emb.convert_chunks_to_vector_index(chunks, vector_index, visualize=True)

Converting ChunkStorage to VectorIndex


100%|██████████| 2660/2660 [00:27<00:00, 95.86it/s] 


In [12]:
answer = vector_index.query(
    emb.embed(
        text="К заявке прилагаются (оригиналы или заверенные в установленном порядке копии):\nсвидетельства о государственной регистрации воздушных судов, заявляемых для полетов в\nцелях авиации общего назначения;\ "
    ), top_k=5
)
print([element.distance for element in answer])
[chunks[element.chunk_storage_key] for element in answer]

[-0.08707942, -0.06643639, -0.06324311, -0.054750953, -0.038149636]


['после травмы с\nограничением OML в течение 3 лет (S06.2; S06.3), перенесших сотрясение или легкую степень ушиба\nголовного мозга - через 3 - 6 месяцев после травмы (S06.0).\n234. При вынесении медицинского заключения учитывается длительность периода измененного\nсознания и амнезии. При прогнозировании возможности развития поздней посттравматической эпилепсии\nкроме характера и степени тяжести травмы учитываются и другие факторы риска (включая\nпсихологические и психические изменения). Оценка проводится с учетом анамнеза по результатам\nрасширенного обследования, включая психологическое тестирование.\n235. Вегетососудистые расстройства, имеющие пароксизмальные проявления заболеваний\nвегетативной нервной системы (ВНС), рецидивирующие обострения и кризовые реакции ваговазального,\nсимпатического, смешанного характера, висцерального, симпаталгического, диэнцефального типа,\nмигрень, солярит, синдром Меньера, диэнцефальный синдром, ангиотрофоневрозы, ортостатическая\nэссенциальная гипото